In [13]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import layers

In [14]:
file_path = "/Users/kiyoshitakeuchi/Desktop/Machine Learning/house-prices-advanced-regression-techniques/"
file_name = "train.csv"
test_file_name = "test.csv"

In [15]:
train_set = pd.read_csv(file_path + file_name)
train_set.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [16]:
label = "SalePrice"
test_set = pd.read_csv(file_path + test_file_name)
test_set.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
5,1466,60,RL,75.0,10000,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
6,1467,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,500,3,2010,WD,Normal
7,1468,60,RL,63.0,8402,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
8,1469,20,RL,85.0,10176,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2010,WD,Normal
9,1470,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,4,2010,WD,Normal


In [17]:
def remove_nan(data_frame): 
    mean = data_frame.select_dtypes(include = 'number').median()
    mean = mean.to_dict()
    data_frame.fillna(value=mean, inplace=True)
    data_frame.fillna(value="?", inplace=True)
    return

remove_nan(test_set)
remove_nan(train_set)

In [18]:
features = train_set.copy()
labels = features.pop(label)

In [19]:
def preprocess_data(data_frame):    
    inputs = {}

    for name, column in data_frame.items():
        dtype = column.dtype
        if dtype == object:
            dtype = tf.string
        else:
            dtype = tf.float32
        inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)
        
    numeric_inputs = {name:input for name,input in inputs.items() 
                      if input.dtype==tf.float32}


    x = layers.Concatenate()(list(numeric_inputs.values()))
    norm = layers.Normalization()
    norm.adapt(np.array(train_set[numeric_inputs.keys()]))
    all_numeric_inputs = norm(x)

    preprocessed_inputs = [all_numeric_inputs]
    
    for name, input in inputs.items():
        if input.dtype == tf.float32:
            continue 

        lookup = layers.StringLookup(vocabulary=np.unique(data_frame[name]))
        one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())
        x = lookup(input)
        x = one_hot(x)
        preprocessed_inputs.append(x)
    
    preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

    preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)
    
    return [inputs, preprocessing]

In [20]:
def model(data_frame):
    body = tf.keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])
    
    inputs, preprocessing_head = preprocess_data(data_frame)
    preprocessed_inputs = preprocessing_head(inputs)
    result = body(preprocessed_inputs)
    model = tf.keras.Model(inputs, result)
    

    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
    return model

In [21]:
features_dict = {name: np.array(value) 
                         for name, value in features.items()}
test_features_dict = {name: np.array(value) 
                         for name, value in test_set.copy().items()}

In [22]:
model = model(features)

In [24]:
model.fit(x=features_dict, y=labels, validation_split = 0.2, epochs=500)

Epoch 1/500
37/37 [==============================] - 1s 16ms/step - loss: 180585.0625 - val_loss: 182228.7188
Epoch 2/500
37/37 [==============================] - 0s 3ms/step - loss: 180539.6719 - val_loss: 182137.3906
Epoch 3/500
37/37 [==============================] - 0s 3ms/step - loss: 180358.0781 - val_loss: 181846.1562
Epoch 4/500
37/37 [==============================] - 0s 3ms/step - loss: 179884.3281 - val_loss: 181177.2656
Epoch 5/500
37/37 [==============================] - 0s 3ms/step - loss: 178906.8594 - val_loss: 179913.0000
Epoch 6/500
37/37 [==============================] - 0s 3ms/step - loss: 177211.1562 - val_loss: 177867.9844
Epoch 7/500
37/37 [==============================] - 0s 3ms/step - loss: 174612.9219 - val_loss: 174868.5469
Epoch 8/500
37/37 [==============================] - 0s 3ms/step - loss: 170928.6562 - val_loss: 170734.3906
Epoch 9/500
37/37 [==============================] - 0s 3ms/step - loss: 165970.3438 - val_loss: 165296.0312
Epoch 10/500
37/37

37/37 [==============================] - 0s 3ms/step - loss: 16275.7812 - val_loss: 20604.7148
Epoch 77/500
37/37 [==============================] - 0s 3ms/step - loss: 16216.9111 - val_loss: 20483.0098
Epoch 78/500
37/37 [==============================] - 0s 3ms/step - loss: 16128.1338 - val_loss: 20461.6523
Epoch 79/500
37/37 [==============================] - 0s 3ms/step - loss: 16073.8301 - val_loss: 20389.9473
Epoch 80/500
37/37 [==============================] - 0s 3ms/step - loss: 15996.4707 - val_loss: 20340.2910
Epoch 81/500
37/37 [==============================] - 0s 3ms/step - loss: 15939.7432 - val_loss: 20254.7148
Epoch 82/500
37/37 [==============================] - 0s 3ms/step - loss: 15904.0615 - val_loss: 20223.2656
Epoch 83/500
37/37 [==============================] - 0s 3ms/step - loss: 15840.8291 - val_loss: 20159.4492
Epoch 84/500
37/37 [==============================] - 0s 3ms/step - loss: 15750.2461 - val_loss: 20127.2734
Epoch 85/500
37/37 [=====================

37/37 [==============================] - 0s 3ms/step - loss: 13789.2959 - val_loss: 18685.3301
Epoch 152/500
37/37 [==============================] - 0s 3ms/step - loss: 13757.2920 - val_loss: 18652.7637
Epoch 153/500
37/37 [==============================] - 0s 3ms/step - loss: 13748.7695 - val_loss: 18640.1738
Epoch 154/500
37/37 [==============================] - 0s 3ms/step - loss: 13738.3408 - val_loss: 18654.6309
Epoch 155/500
37/37 [==============================] - 0s 3ms/step - loss: 13736.2998 - val_loss: 18632.0195
Epoch 156/500
37/37 [==============================] - 0s 3ms/step - loss: 13771.0010 - val_loss: 18631.0195
Epoch 157/500
37/37 [==============================] - 0s 3ms/step - loss: 13711.7725 - val_loss: 18622.2832
Epoch 158/500
37/37 [==============================] - 0s 3ms/step - loss: 13696.6182 - val_loss: 18614.9258
Epoch 159/500
37/37 [==============================] - 0s 3ms/step - loss: 13666.0020 - val_loss: 18586.7734
Epoch 160/500
37/37 [============

37/37 [==============================] - 0s 3ms/step - loss: 13163.9951 - val_loss: 18222.5820
Epoch 227/500
37/37 [==============================] - 0s 3ms/step - loss: 13128.6631 - val_loss: 18160.8242
Epoch 228/500
37/37 [==============================] - 0s 3ms/step - loss: 13119.5586 - val_loss: 18145.4746
Epoch 229/500
37/37 [==============================] - 0s 3ms/step - loss: 13118.9258 - val_loss: 18167.2227
Epoch 230/500
37/37 [==============================] - 0s 3ms/step - loss: 13098.4619 - val_loss: 18139.1582
Epoch 231/500
37/37 [==============================] - 0s 3ms/step - loss: 13093.1895 - val_loss: 18141.6602
Epoch 232/500
37/37 [==============================] - 0s 3ms/step - loss: 13085.2334 - val_loss: 18131.5195
Epoch 233/500
37/37 [==============================] - 0s 3ms/step - loss: 13092.7656 - val_loss: 18124.9258
Epoch 234/500
37/37 [==============================] - 0s 3ms/step - loss: 13088.7783 - val_loss: 18182.1602
Epoch 235/500
37/37 [============

37/37 [==============================] - 0s 3ms/step - loss: 12815.4502 - val_loss: 17899.7656
Epoch 302/500
37/37 [==============================] - 0s 3ms/step - loss: 12817.4287 - val_loss: 17926.6602
Epoch 303/500
37/37 [==============================] - 0s 3ms/step - loss: 12808.5352 - val_loss: 17895.0762
Epoch 304/500
37/37 [==============================] - 0s 3ms/step - loss: 12803.5498 - val_loss: 17934.9629
Epoch 305/500
37/37 [==============================] - 0s 3ms/step - loss: 12796.8584 - val_loss: 17953.6992
Epoch 306/500
37/37 [==============================] - 0s 3ms/step - loss: 12795.9062 - val_loss: 17944.5801
Epoch 307/500
37/37 [==============================] - 0s 3ms/step - loss: 12796.3105 - val_loss: 17979.4414
Epoch 308/500
37/37 [==============================] - 0s 3ms/step - loss: 12803.8643 - val_loss: 17907.5625
Epoch 309/500
37/37 [==============================] - 0s 3ms/step - loss: 12786.6846 - val_loss: 17922.5215
Epoch 310/500
37/37 [============

37/37 [==============================] - 0s 3ms/step - loss: 12537.1924 - val_loss: 17807.5156
Epoch 377/500
37/37 [==============================] - 0s 3ms/step - loss: 12572.2793 - val_loss: 17779.4238
Epoch 378/500
37/37 [==============================] - 0s 3ms/step - loss: 12529.9238 - val_loss: 17806.5566
Epoch 379/500
37/37 [==============================] - 0s 3ms/step - loss: 12519.3428 - val_loss: 17762.7852
Epoch 380/500
37/37 [==============================] - 0s 3ms/step - loss: 12529.1465 - val_loss: 17822.7520
Epoch 381/500
37/37 [==============================] - 0s 3ms/step - loss: 12511.3584 - val_loss: 17849.5566
Epoch 382/500
37/37 [==============================] - 0s 3ms/step - loss: 12512.2520 - val_loss: 17821.5078
Epoch 383/500
37/37 [==============================] - 0s 3ms/step - loss: 12534.2422 - val_loss: 17786.9355
Epoch 384/500
37/37 [==============================] - 0s 4ms/step - loss: 12539.2803 - val_loss: 17804.5781
Epoch 385/500
37/37 [============

37/37 [==============================] - 0s 3ms/step - loss: 12286.3779 - val_loss: 17749.5898
Epoch 452/500
37/37 [==============================] - 0s 3ms/step - loss: 12258.9756 - val_loss: 17779.1133
Epoch 453/500
37/37 [==============================] - 0s 3ms/step - loss: 12307.6689 - val_loss: 17723.6973
Epoch 454/500
37/37 [==============================] - 0s 3ms/step - loss: 12265.4199 - val_loss: 17728.5898
Epoch 455/500
37/37 [==============================] - 0s 3ms/step - loss: 12278.8164 - val_loss: 17713.5352
Epoch 456/500
37/37 [==============================] - 0s 3ms/step - loss: 12266.7139 - val_loss: 17787.7402
Epoch 457/500
37/37 [==============================] - 0s 3ms/step - loss: 12274.4365 - val_loss: 17712.5977
Epoch 458/500
37/37 [==============================] - 0s 3ms/step - loss: 12253.5234 - val_loss: 17730.4434
Epoch 459/500
37/37 [==============================] - 0s 3ms/step - loss: 12239.6406 - val_loss: 17749.3008
Epoch 460/500
37/37 [============

In [25]:
array = model.predict(test_features_dict)

In [26]:
import csv

with open('regression_solution_2.csv', 'w', newline='') as file:
    fieldnames = ['Id', 'SalePrice']
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    n = 1461
    for element in array:
        writer.writerow({'Id': n, 'SalePrice': element[0]})
        n += 1